In [7]:
%pip install langchain-groq langchain sentence-transformers langchain-community pypdf chromadb
%pip install gdown

In [2]:
#Importing Libraries
from sentence_transformers import SentenceTransformer
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.storage import InMemoryStore
from langchain_groq import ChatGroq


In [9]:
import gdown


In [11]:
file_id = "1O_H4rHmf2YBj9TYaSj9ovrV1npprpqe-"
url = f"https://drive.google.com/uc?id={file_id}"
output = "The_Constitution_of_Kenya_2010.pdf"

gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1O_H4rHmf2YBj9TYaSj9ovrV1npprpqe-
To: /content/The_Constitution_of_Kenya_2010.pdf
100%|██████████| 1.72M/1.72M [00:00<00:00, 132MB/s]


'The_Constitution_of_Kenya_2010.pdf'

In [12]:
loader=PyPDFLoader(output)
docs=loader.load()
print(len(docs))

193


In [13]:
#initializing parent and text splitters
parent_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100
)

child_splitter=RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=50
)

In [14]:
#initializing the embedding model
bge_model=SentenceTransformer("BAAI/bge-base-en-v1.5")
class BGEEmbeddings:
  def embed_documents(self, text):
    return bge_model.encode(text,batch_size=8,normalize_embeddings=True).tolist()
  def embed_query(self, query):
    return bge_model.encode([query],normalize_embeddings=True).tolist()[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
#creating the vector store
cons_vector_store=Chroma(
    collection_name="constitution",
    embedding_function=BGEEmbeddings(),
    persist_directory="./chroma",
)

/tmp/ipython-input-975094595.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  cons_vector_store=Chroma(


In [16]:
#creating in-memory store
store=InMemoryStore()
#creating retrievers
retriever=ParentDocumentRetriever(
    vectorstore=cons_vector_store,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [17]:
#adding documents into the vector store
retriever.add_documents(docs)

In [18]:
from google.colab import userdata
userdata.get('constitution')

'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'

In [19]:
#creating a child retriever
child_retriever=cons_vector_store.as_retriever()

In [20]:
#initializing the LLM
llm=ChatGroq(
   groq_api_key=userdata.get('constitution'),
   model_name="llama-3.3-70b-versatile",
   temperature=0.7

)

In [21]:
#creating the template
template="""
Answer the following questions according to the context given.
If the question is out of context,just say the question is out of context,just give me another question.
Do not try to answer a question out of context.
{context}
Question:{question}
"""

In [22]:
#creating the chain
qa_chain=PromptTemplate(template=template,input_variable=["context","question"])

In [23]:
chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":qa_chain},
    return_source_documents=True
)

In [24]:
question="What is the role of Parliament according to the Constitution of Kenya, 2010?"
answer=chain({"query":question})
print(answer["result"])

/tmp/ipython-input-2601425803.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer=chain({"query":question})


The role of Parliament according to the Constitution of Kenya, 2010, is derived from Article 94, which states that the legislative authority of the Republic is derived from the people and is exercised at the national and county levels. Specifically, the National Assembly: 

- exercises legislative authority (Article 94)
- exercises oversight of State organs (Article 94)
- approves declarations of war and extensions of states of emergency (Article 6)

Additionally, the Senate represents the counties, protects their interests, participates in law-making, determines the allocation of national revenue among counties, and exercises oversight over national revenue allocated to county governments (Article 96). 

In general, Parliament performs its functions in accordance with the Constitution, and its role includes law-making, oversight, and representation of the people.


In [25]:
question="What are the three branches of government created by the Constitution of Kenya?"
answer=chain({"query":question})
print(answer["result"])

The three branches of government created by the Constitution of Kenya are:

1. The Legislative: This includes Parliament and the legislative assemblies in the county governments.
2. The Executive: This includes the national executive and the executive structures in the county governments.
3. The Judiciary: This includes the Judiciary and independent tribunals.

These branches are mentioned in Article 1(3) of the Constitution, which states that sovereign power is delegated to these State organs.


In [26]:
question="Can the constitution of Kenya be altered? if so,how?"
answer=chain({"query":question})
print(answer["result"])

Yes, the Constitution of Kenya can be altered. According to the provided context, the people of Kenya have the sovereign right to replace the Constitution, and the Constitution of Kenya Review Act, 2008, provided a legal framework for the comprehensive review and replacement of the current Constitution. 

The alteration process involves the people of Kenya participating in generating and debating proposals to alter the Constitution, which should result in a new Constitution that faithfully reflects the wishes of the people of Kenya. 

Additionally, the context mentions that the Constitution of Kenya (Amendment) Act, 2008, and the Constitution of Kenya Review Acts of 1997 and 2008, provided a framework for the review process, which included accommodating the diversity of the Kenyan people and respecting universal principles of human rights, gender equity, and democracy.

However, the specific procedures for altering the Constitution are not detailed in the provided context. It is mentio

In [27]:
question="Is it constitutional to eat ice cream for breakfast?"
answer=chain({"query":question})
print(answer["result"])

The question is out of context. 

Here's another question: What is the significance of Article 45 of the Constitution of Kenya, 2010, in relation to the family?


### Creating a ChatBot
### Interface using Gradio

In [28]:
def qa_bot(message, history):
    # If history is None, initialize it as an empty list
    history = history or []

    try:
        # Running the RAG chain with the user's query (message)
        result = chain({"query": message})  #Getting the answer from the result dictionary
        answer = result.get("result") or result.get("answer") or "Sorry, no answer found." # If 'result' key is missing, fall back to 'answer'; if both fail, use default message
        source_docs = result.get("source_documents", []) # Getting the list of source documents used to generate the answer

        # If no documents retrieved, assume the question is out-of-context
        if not source_docs:
            reply = "Sorry, I could not find an answer in the document. Please ask another question related to the document."
            history.append({"role": "user", "content": message}) # Adding user message and assistant reply to the chat history
            history.append({"role": "assistant", "content": reply})
            return history

        # If source documents are found, get the source metadata of the first one
        source = source_docs[0].metadata.get("source", "Unknown source")
        full_answer = f"{answer} from: {source}"  # Combining the answer with the source info to form the full reply
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": full_answer})
        return history # Return the updated chat history to Gradio
    except Exception as e:
        # If something goes wrong, catch the error
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": f"Error: {str(e)}"})
        return history


In [29]:
!pip install --upgrade gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 25.5 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.12.1
    Uninstalling gradio_client-1.12.1:
      Successfully uninstalled gradio_client-1.12.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.44.1
    Uninstalling gradio-5.44.1:
      Successfully uninstalled gradio-5.44.1


In [30]:
import gradio as gr



In [31]:
import gradio as gr

# ChatInterface version
demo = gr.ChatInterface(
    fn=qa_bot,
    title="CONSTITUTION OF KENYA (RAG CHATBOT)",
    description="Ask questions based on the Constitution of Kenya document. Out-of-context questions will be politely rejected.",
    examples=[
        "What are the three branches of government created by the Constitution of Kenya?",
        "Can the constitution of Kenya be altered? If so, how?",
        "(Out of context Question) :Is it constitutionally wrong to eat ice cream for breakfast?"
    ],
    type="messages"
)

demo.launch(share=True)




Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2c72c601df414f727.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
